In [1]:
import os
import requests
import pandas as pd
import numpy as np
import datetime
import glob


In [2]:
ISP1Requirements = 'ISP1Requirements'
ISP1UnitAvailabilities = 'ISP1UnitAvailabilities'
aggr_curve = 'aggr_curve'

In [3]:
cur_wd = os.getcwd()
Requirements_path = cur_wd+"/"+ ISP1Requirements +"/"
Unit_path =  cur_wd+"/"+ ISP1UnitAvailabilities +"/"
curve_path =  cur_wd+"/"+ aggr_curve +"/"

In [4]:
requirements_files = glob.glob(os.path.join(Requirements_path, "*.xlsx"))


In [5]:
import warnings
warnings.filterwarnings('ignore')
requirements = dict()
for f in requirements_files:
    date = f.split('/')[-1].split()[0].split('_')[0]

    # read the csv file
    df = pd.read_excel(f)

    RES = df.loc[3]
    Load = df.loc[7]
    Hydro = df.loc[27]
    
    temp = pd.concat([RES, Load, Hydro], axis=1)
    temp.index = df.loc[0]
    temp.rename(columns = {3:'RES forecast',7:'Load forecast', 27:'Mandatory Hydro'}, inplace=True)
    requirements[date] = temp[1:-1].dropna()

In [6]:
timehrlist = list()
for i in range(24):
    timehrlist.append( "%02d" % (i,)+':30' )

for name, df in requirements.items():
    df = df.drop(labels = timehrlist, axis = 0, errors = 'ignore')
    df = df.dropna()
    df = df.reset_index()
    df.rename(columns={0:"Hour"},inplace=True)
    df['date_time'] = name + ' '+ df['Hour']
    df = df.set_index('date_time')
    requirements[name] = df
    requirements[name]['Residual Load'] = df['Load forecast'] - df['RES forecast'] - df['Mandatory Hydro'] 

In [7]:
requirements['20220120']



,Hour,RES forecast,Load forecast,Mandatory Hydro,Residual Load
date_time,,,,,
20220120 00:00,00:00,1030,5487,0,4457
20220120 01:00,01:00,1130,5366,0,4236
20220120 02:00,02:00,1210,5270,0,4060
20220120 03:00,03:00,1270,5135,50,3815
20220120 04:00,04:00,1320,5161,50,3791
20220120 05:00,05:00,1360,5593,225,4008
20220120 06:00,06:00,1440,6398,426,4532
20220120 07:00,07:00,1670,7075,616,4789
20220120 08:00,08:00,2340,7537,598,4599


Data are stored in requirements

Next: create df's that have the same index, then merge the tables by index 

In [8]:
unit_files = glob.glob(os.path.join(Unit_path, "*.xlsx"))

In [9]:
units = dict()
lignite_unitnames = ['AG_DIMITRIOS1', 'AG_DIMITRIOS2', 'AG_DIMITRIOS3', 'AG_DIMITRIOS4', 'AG_DIMITRIOS5', 'KARDIA3', 'KARDIA4', 'MEGALOPOLI3', 'MEGALOPOLI4', 'MELITI']
natural_gas_unitnames = ['ALIVERI5', 'LAVRIO4', 'LAVRIO5', 'KOMOTINI', 'MEGALOPOLI_V', 'HERON1', 'HERON2', 'HERON3', 'HERON_CC', 'ELPEDISON_THESS', 'ELPEDISON_THISVI', 'ALOUMINIO', 'PROTERGIA_CC', 'KORINTHOS_POWER']
hydro_unitnames = ['KREMASTA', 'KASTRAKI', 'STRATOS1', 'ILARIONAS', 'POLYFYTO', 'SFIKIA', 'ASOMATA', 'THESAVROS1', 'THESAVROS2', 'THESAVROS3', 'PLATANOVRYSI', 'POURNARI1', 'POURNARI2', 'P_AOOU', 'LADONAS', 'PLASTIRAS', 'EDESSAIOS', 'AGRAS']
import warnings
warnings.filterwarnings('ignore')

for f in unit_files:
    date = f.split('/')[-1].split()[0].split('_')[0]
    # read the csv file
    df = pd.read_excel(f)
    df = df.dropna()
    df = df.iloc[:,[1,3]]
    tot = df.iloc[1:,1].sum()
    avai_lignite = df[df.iloc[:,0].isin(lignite_unitnames)].iloc[:,1].sum()
    avai_natural_gas=df[df.iloc[:,0].isin(natural_gas_unitnames)].iloc[:,1].sum()
    avai_hydro = df[df.iloc[:,0].isin(hydro_unitnames)].iloc[:,1].sum()
    requirements[date]['avai_lignite'] = avai_lignite
    requirements[date]['avai_natural_gas'] = avai_natural_gas
    requirements[date]['avai_hydro'] = avai_hydro


In [10]:
requirements['20220120']


,Hour,RES forecast,Load forecast,Mandatory Hydro,Residual Load,avai_lignite,avai_natural_gas,avai_hydro
date_time,,,,,,,,
20220120 00:00,00:00,1030,5487,0,4457,1343,3637,2156
20220120 01:00,01:00,1130,5366,0,4236,1343,3637,2156
20220120 02:00,02:00,1210,5270,0,4060,1343,3637,2156
20220120 03:00,03:00,1270,5135,50,3815,1343,3637,2156
20220120 04:00,04:00,1320,5161,50,3791,1343,3637,2156
20220120 05:00,05:00,1360,5593,225,4008,1343,3637,2156
20220120 06:00,06:00,1440,6398,426,4532,1343,3637,2156
20220120 07:00,07:00,1670,7075,616,4789,1343,3637,2156
20220120 08:00,08:00,2340,7537,598,4599,1343,3637,2156


In [11]:
requirements['20220121']



,Hour,RES forecast,Load forecast,Mandatory Hydro,Residual Load,avai_lignite,avai_natural_gas,avai_hydro
date_time,,,,,,,,
20220121 00:00,00:00,1650,5324,0,3674,1343,4057,2156
20220121 01:00,01:00,1660,5239,0,3579,1343,4057,2156
20220121 02:00,02:00,1680,5129,0,3449,1343,4057,2156
20220121 03:00,03:00,1750,5041,0,3291,1343,4057,2156
20220121 04:00,04:00,1800,5069,0,3269,1343,4057,2156
20220121 05:00,05:00,1860,5525,251,3414,1343,4057,2156
20220121 06:00,06:00,1880,6454,446,4128,1343,4057,2156
20220121 07:00,07:00,2000,7059,661,4398,1343,4057,2156
20220121 08:00,08:00,2420,7400,739,4241,1343,4057,2156


In [12]:
curve_files = glob.glob(os.path.join(curve_path, "*.xlsx"))

In [13]:
timehrlist = list()
for i in range(24):
    timehrlist.append( "%02d" % (i,)+':00' )

from datetime import datetime
for f in curve_files:
    date = f.split('/')[-1].split('.')[0]

    # read the csv file
    df = pd.read_excel(f)
    df = df.dropna()
    df = df[df['SIDE_DESCR'] == 'Sell']
    df['DELIVERY_MTU'] = pd.to_datetime(df['DELIVERY_MTU'])
    df['DELIVERY_MTU'] = df['DELIVERY_MTU'].dt.strftime("%Y%m%d %H:%M")

    # now that we have df containing each price-quantity at each hour, we need to combine them into a list of tuple for each hr
    # then put this list into the corresponding row of a new column in requirements[date] 
    q = list()
    p = list()
    for t in timehrlist:
        q.append(df[df['DELIVERY_MTU']== date + " "+t]['QUANTITY'].tolist())
        p.append(df[df['DELIVERY_MTU']== date + " "+t]['UNITPRICE'].tolist())

    requirements[date]['QUANTITY']=q
    q_15 = list()
    q_15 = list()
    q_55 = list()
    q_60 = list()
    q_65 = list()
    q_70 = list()
    q_80 = list()
    q_85 = list()
    q_90 = list()
    q_95 = list()
    q_100 = list()
    
    q_25 = list()
    q_50 = list()
    q_75 = list()
    for i in q:
        q_15.append(np.percentile(i,15,interpolation = 'lower'))
        q_55.append(np.percentile(i,55,interpolation = 'lower'))
        q_60.append(np.percentile(i,60,interpolation = 'lower'))
        q_65.append(np.percentile(i,65,interpolation = 'lower'))
        q_70.append(np.percentile(i,70,interpolation = 'lower'))
        q_25.append(np.percentile(i,25,interpolation = 'lower'))
        q_50.append(np.percentile(i,50,interpolation = 'lower'))
        q_75.append(np.percentile(i,75,interpolation = 'lower'))
        q_80.append(np.percentile(i,80,interpolation = 'lower'))
        q_85.append(np.percentile(i,85,interpolation = 'lower'))
        q_90.append(np.percentile(i,90,interpolation = 'lower'))
        q_95.append(np.percentile(i,95,interpolation = 'lower'))
        q_100.append(np.percentile(i,100,interpolation = 'lower'))
    requirements[date]['q_15'] = q_15
    requirements[date]['q_55'] = q_55
    requirements[date]['q_60'] = q_60
    requirements[date]['q_65'] = q_65
    requirements[date]['q_70'] = q_70
    requirements[date]['q_25'] = q_25
    requirements[date]['q_50'] = q_50
    requirements[date]['q_75'] = q_75
    requirements[date]['q_80'] = q_80
    requirements[date]['q_85'] = q_85
    requirements[date]['q_90'] = q_90
    requirements[date]['q_95'] = q_95
    requirements[date]['q_100'] = q_100
    requirements[date]['UNITPRICE']=p
    p_15 = list()
    p_15 = list()
    p_55 = list()
    p_60 = list()
    p_65 = list()
    p_70 = list()
    p_25 = list()
    p_50 = list()
    p_75 = list()
    p_80 = list()
    p_85 = list()
    p_90 = list()
    p_95 = list()
    p_100 = list()
    for i in p:
        p_15.append(np.percentile(i,15,interpolation = 'lower'))
        p_55.append(np.percentile(i,55,interpolation = 'lower'))
        p_60.append(np.percentile(i,60,interpolation = 'lower'))
        p_65.append(np.percentile(i,65,interpolation = 'lower'))
        p_70.append(np.percentile(i,70,interpolation = 'lower'))
        p_25.append(np.percentile(i,25,interpolation = 'lower'))
        p_50.append(np.percentile(i,50,interpolation = 'lower'))
        p_75.append(np.percentile(i,75,interpolation = 'lower'))
        p_80.append(np.percentile(i,80,interpolation = 'lower'))
        p_85.append(np.percentile(i,85,interpolation = 'lower'))
        p_90.append(np.percentile(i,90,interpolation = 'lower'))
        p_95.append(np.percentile(i,95,interpolation = 'lower'))
        p_100.append(np.percentile(i,100,interpolation = 'lower'))
    requirements[date]['p_15'] = p_15
    requirements[date]['p_55'] = p_55
    requirements[date]['p_60'] = p_60
    requirements[date]['p_65'] = p_65
    requirements[date]['p_70'] = p_70
    requirements[date]['p_25'] = p_25
    requirements[date]['p_50'] = p_50
    requirements[date]['p_75'] = p_75
    requirements[date]['p_80'] = p_80
    requirements[date]['p_85'] = p_85
    requirements[date]['p_90'] = p_90
    requirements[date]['p_95'] = p_95
    requirements[date]['p_100'] = p_100
#     display(requirements[date].iloc[1]['Hour'])

    # load - res - hydro = residual load
    # margin: diff(residual load), takes into account the ability of plants in reacting to demand changes
    

In [14]:
# construct polynomial base on quantity and unit price
# how? use a model to predict and output the coefficients?

In [15]:
# merge data from all days
final_df = pd.concat(requirements.values(), ignore_index=False)

In [16]:
final_df = final_df.sort_values(by = 'date_time')

In [17]:
final_df['Residual Load Diff'] = final_df['Residual Load'].diff()

I have a quesiton:
    You mentioned how calculating a difference of residual load 'diff' would be benefitial to getting the bidding curve.
    
    I began to ponder on how exactly this could help, and came up with some explanation: Under the circumstances 
    where the bidding curve of the previous hour is known (and these 2 times are in the same day), the load is the only factor that
    has changed, and 'diff' by definition is the most straight foward representation of this change.
    
    Thus I wonder, in our model training, are we suppose to provide the bidding curve/result of the previous hour?
    I don't see any problem with it UNLESS in real life scenario, these bidding curve aren't available information to the bidders.

In [18]:
final_df.to_excel("fulldata.xlsx")

In [19]:
final_df[final_df['p_25'] < 0]

,Hour,RES forecast,Load forecast,Mandatory Hydro,Residual Load,avai_lignite,avai_natural_gas,avai_hydro,QUANTITY,q_15,...,p_55,p_60,p_65,p_70,p_25,p_50,p_75,p_80,p_85,Residual Load Diff
date_time,,,,,,,,,,,,,,,,,,,,,
